In [ ]:
from extraction import extract_articles_from_google
from extraction import extract_data_from_page
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By

In [ ]:
pages_file_name = "cnn_español_articles.json"
content_file_name = "cnn_español_articles_with_content.json"

In [ ]:
extract_articles_from_google(
    website="cnnespanol.cnn.com",
    newscast="CNN en Español",
    pages=2,
    output=pages_file_name,
    keywords="Estallido Social",
    )

In [ ]:
def get_content(body: WebElement):
	content = ""
	for element in body.find_element(By.CLASS_NAME, "article__content").find_elements(By.TAG_NAME, "p"):
		content += element.text.strip() + "\n"
	return content

def get_author(body: WebElement):
	try:
		return body.find_element(By.CLASS_NAME, "vossi-byline__names").text.split("Por")[1].strip()
	except Exception:
		return "The website does not provide an author"

def get_description(body: WebElement):
    return "The website does not provide a description"

extract_data_from_page(
	input_file=pages_file_name, 
	output_file=content_file_name,
	get_author=get_author,
	get_description=get_description,
	get_content=get_content,
	limit_of_pages=5,
	wait=60,
)